# Gluon Model Evaluation

In [1]:
# Import necessary Libraries
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd, gluon
import numpy as np
import h5py
from sklearn.metrics import accuracy_score

/home/ec2-user/anaconda3/envs/mxnet_p27/lib/python2.7/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


### Datasets

In [2]:
# Import the Datasets
dataset = h5py.File('datasets/datasets.h5', 'r')
train_set_x_orig = np.array(dataset['train_set_x'][:])
train_set_y_orig = np.array(dataset['train_set_y'][:])
test_set_x_orig = np.array(dataset['test_set_x'][:])
test_set_y_orig = np.array(dataset['test_set_y'][:])

In [3]:
# View Training Dataset
print("Training Data Dimensions: {}".format(train_set_x_orig.shape))
print("No. Examples: {}".format(train_set_x_orig.reshape((-1, 12288)).shape[0]))
print("No. Inputs: {}".format(train_set_x_orig.reshape((-1, 12288)).shape[1]))

Training Data Dimensions: (209, 64, 64, 3)
No. Examples: 209
No. Inputs: 12288


In [4]:
# View Testing Dataset
print("Testing Data Dimensions: {}".format(test_set_x_orig.shape))
print("No. Examples: {}".format(test_set_x_orig.reshape((-1, 12288)).shape[0]))
print("No. Inputs: {}".format(test_set_x_orig.reshape((-1, 12288)).shape[1]))

Testing Data Dimensions: (50, 64, 64, 3)
No. Examples: 50
No. Inputs: 12288


In [5]:
# Pre-process Data
def transform(x):
    v = x.reshape((x.shape[0], (x.shape[1] * x.shape[2]) * x.shape[3]))
    return v.astype(np.float32) / 255
train_data = transform(train_set_x_orig)
train_label = train_set_y_orig.astype(np.float32)
test_data = transform(test_set_x_orig)
test_label = test_set_y_orig.astype(np.float32)
print("Final Training Data Dimensions: {}".format(train_data.shape))
print("Final Testing Data Dimensions: {}".format(test_data.shape))

Final Training Data Dimensions: (209, 12288)
Final Testing Data Dimensions: (50, 12288)


### Neural Network Configuration

In [6]:
# Configure the Dataset
batch_size = 64 # Using full set of observations
num_examples = 209
net_dims = [20, 7, 5, 1]
lr = 0.0075

# Create Training and Test Data Iterator
train_data = mx.gluon.data.DataLoader(
    mx.gluon.data.ArrayDataset(
        train_data,
        train_label
    ),
    shuffle=True,
    batch_size=batch_size
)
test_data = mx.gluon.data.DataLoader(
    mx.gluon.data.ArrayDataset(
        test_data,
        test_label
    ),
    shuffle=False,
    batch_size=batch_size
)

### Neural Network Model

In [7]:
# Network Model
model_ctx = mx.cpu()
net = gluon.nn.HybridSequential()
with net.name_scope():
    net.add(gluon.nn.Dense(net_dims[0], activation='relu'))
    net.add(gluon.nn.Dense(net_dims[1], activation='relu'))
    net.add(gluon.nn.Dense(net_dims[2], activation='relu'))
    net.add(gluon.nn.Dense(net_dims[3], activation='sigmoid'))
net.hybridize()

# Parameter Initialization
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24))
# Optimizer
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
# Cross Entropy Loss Function
binary_ce = gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=True)

# Evlauation metric
def eval_acc(data_iterator, net):
    acc = mx.metric.Accuracy()
    acc.reset()
    for i, (data, label) in enumerate(data_iterator):
        #acc.reset()
        data = data.as_in_context(model_ctx)
        label = label.as_in_context(model_ctx)
        output = net(data)
        #predictions = nd.argmax(output, axis=1)
        predictions = (nd.sign(output) + 1) / 2
        #num_correct = nd.sum(predictions == label)
        acc.update(preds=output, labels=label)
    return acc.get()[1]
    #return num_correct.asscalar()/209

In [8]:
# Accuracy Score
def accuracy(data_iterator, net, Y):
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(model_ctx)
        label = label.as_in_context(model_ctx)
        output = net(data)
        decision_boundary = np.vectorize(lambda x: 1 if x > 0.5 else 0)
        y_pred = list(decision_boundary(output.asnumpy()).flat)
        Y = list(Y.flat)
        
    return accuracy_score(Y, y_pred)

### Model Training

In [ ]:
# Training Loop
epochs = 2500
costs = []

for e in range(epochs):
    cumulative_loss = 0
    # Enumerate bacthes
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(model_ctx)
        label = label.as_in_context(model_ctx)
        # Record for calculating derivatives for forward pass
        with autograd.record():
            output = net(data)
            loss = binary_ce(output, label)
        # Run backward pass
        loss.backward()
        trainer.step(data.shape[0])
        cumulative_loss += nd.sum(loss).asscalar()
    test_accuracy = accuracy(test_data, net, test_set_y_orig)
    #train_accuracy = accuracy(train_data, net, train_set_y_orig)
    costs.append(cumulative_loss/num_examples)
    if e % 100 ==0:
        #print("Epoch: {}; Loss: {}; Training Acc: {}; Testing Acc: {}"\
        #      .format(e,cumulative_loss/num_examples,train_accuracy,test_accuracy))
        print("Epoch: {}; Loss: {}; Test Set Accuracy: {}"\
              .format(e,cumulative_loss/num_examples,test_accuracy))

### Training Analysis

In [ ]:
# Final Score
final_accuracy = accuracy(test_data, net, test_set_y_orig)
print("Epoch: {}\nLoss: {}\nTest Set Accuracy: {}%".format(e,cumulative_loss/num_examples, final_accuracy*100))

# Plot the convergence of the cost function
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.figure(num=None,figsize=(8, 6))
plt.plot(costs)
plt.grid(True, which="both")
plt.xlabel('Epoch',fontsize=14)
plt.ylabel('Cost',fontsize=14)
plt.title("Learning Rate: " + str(lr))
plt.show;

### Predict on Unseen Data

In [ ]:
# Predict Funciton
def predict(image, nn):
    data = nd.array(image).as_in_context(model_ctx)
    output = nn(data)
    prediction = (nd.sign(output) + 1) /2
    return prediction

#### Grumpy Cat

In [ ]:
# Display and predict against grumpy cat image
from skimage import transform
classes = ['non-cat', 'cat']
my_image = "grumpy_cat.jpeg"
#y = [1] # the true class label of the image (1 -> cat)
fname = "images/" + my_image
# Read the image
img = plt.imread(fname)
# Pre-process the image
image = transform.resize(img, (64, 64), mode='constant').reshape((1, 64 * 64 * 3))
# Run the model against the image
my_predicted_image = np.squeeze(predict(image, net).asnumpy()[0])
# Plot the final image
plt.rcParams['figure.figsize'] = (6.0, 7.0)
plt.imshow(img);
# Print the results
print ("y = " + str(np.squeeze(my_predicted_image)) + \
       "\nThe trained model predicts a \"" + \
       classes[int(my_predicted_image)] +  "\" picture.")

#### Gargouille

In [ ]:
# Display and predict against the gargouille image
my_image = "gargouille.jpeg"
#y = [1] # the true class label of the image (1 -> cat)
fname = "images/" + my_image
# Read the image
img = plt.imread(fname)
# Pre-process the image
image = transform.resize(img, (64, 64), mode='constant').reshape((1, 64 * 64 * 3))
# Run the model against the image
my_predicted_image = np.squeeze(predict(image, net).asnumpy()[0])
# Plot the final image
plt.rcParams['figure.figsize'] = (6.0, 7.0)
plt.imshow(img);
# Print the results
print ("y = " + str(np.squeeze(my_predicted_image)) + \
       "\nThe trained model predicts a \"" + \
       classes[int()] +  "\" picture.")

### Save the Model and Optmized Parameters

In [ ]:
# Create aplaceholder
z = net(mx.sym.var('data'))

# Save the model
z.save('model.json')

# Save the Optimized Parameters
net.collect_params().save('model.params')

### Load the Model and Test

In [ ]:
symbol = mx.sym.load('model.json')
outputs = mx.sym.sigmoid(data=symbol, name='sigmoid_label')
inputs = mx.sym.var('data')
param_dict = gluon.ParameterDict('model_')
model = gluon.SymbolBlock(outputs, inputs, param_dict)
model.load_params('model.params', ctx=model_ctx)

In [ ]:
# Display and predict against grumpy cat image
from skimage import transform
classes = ['non-cat', 'cat']
my_image = "grumpy_cat.jpeg"
#y = [1] # the true class label of the image (1 -> cat)
fname = "images/" + my_image
# Read the image
img = plt.imread(fname)
# Pre-process the image
image = transform.resize(img, (64, 64), mode='constant').reshape((1, 64 * 64 * 3))
# Run the model against the image
my_predicted_image = np.squeeze(predict(image, model).asnumpy()[0])
# Plot the final image
plt.rcParams['figure.figsize'] = (6.0, 7.0)
plt.imshow(img);
# Print the results
print ("y = " + str(np.squeeze(my_predicted_image)) + \
       "\nThe trained model predicts a \"" + \
       classes[int(my_predicted_image)] +  "\" picture.")

In [ ]:
# Display and predict against the gargouille image
my_image = "gargouille.jpeg"
#y = [1] # the true class label of the image (1 -> cat)
fname = "images/" + my_image
# Read the image
img = plt.imread(fname)
# Pre-process the image
image = transform.resize(img, (64, 64), mode='constant').reshape((1, 64 * 64 * 3))
# Run the model against the image
my_predicted_image = np.squeeze(predict(image, model).asnumpy()[0])
# Plot the final image
plt.rcParams['figure.figsize'] = (6.0, 7.0)
plt.imshow(img);
# Print the results
print ("y = " + str(np.squeeze(my_predicted_image)) + \
       "\nThe trained model predicts a \"" + \
       classes[int()] +  "\" picture.")

In [ ]:
# Training Loop
epochs = 25

for e in range(epochs):
    cumulative_loss = 0
    # Enumerate bacthes
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(model_ctx)
        label = label.as_in_context(model_ctx)
        # Record for calculating derivatives for forward pass
        with autograd.record():
            output = net(data)
            loss = binary_ce(output, label)
        # Run backward pass
        loss.backward()
        trainer.step(data.shape[0])
        current_loss = nd.mean(loss).asscalar()
    print("Epoch: {}; Loss: {}".format(e,current_loss))

In [ ]:
y_pred = np.array([])
for data, label in test_data:
    data = data.as_in_context(mx.cpu()).astype('float32')
    label = label.as_in_context(mx.cpu()).astype('float32')
    output = net(data)
    y_pred = np.append(y_pred, output.asnumpy())

y_pred_labels = np.where(y_pred > 0.5, 1, 0)
print(accuracy_score(test_set_y_orig, y_pred_labels))

In [ ]:
num_correct = 0.0
num_total = 50
for data, label in test_data:
    data = data.as_in_context(mx.cpu()).astype('float32')
    label = label.as_in_context(mx.cpu()).astype('float32')
    output = net(data)
    prediction = (nd.sign(output) + 1) / 2
    num_correct += nd.sum(prediction == label)
print("Accuracy: %0.3f (%s/%s)" % (num_correct.asscalar()/num_total, num_correct.asscalar(), num_total))

In [11]:
metric = mx.metric.Accuracy()
def test(ctx, net, val_data):
    for data, label in val_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        metric.update([label], [output])
    return metric.get()

ctx = mx.cpu()
epochs = 2500
for e in range(epochs):
    metric.reset()
    for data, label in train_data:
        data = data.as_in_context(mx.cpu())
        label = label.as_in_context(mx.cpu())
        # Record for calculating derivatives for forward pass
        with autograd.record():
            output = net(data)
            loss = binary_ce(output, label)
            # Run backward pass
            loss.backward()
        trainer.step(data.shape[0])
        current_loss = nd.mean(loss).asscalar()
        metric.update([label], [output])
    
    if e % 100 == 0:
        name, acc = metric.get()
        print("Epoch: {}; Loss: {}; Training Acc: {}{}" .format(e,current_loss, name, acc))
        name, val_acc = test(ctx, net, test_data)
        print("Teting Acc: {}{}".format(name, val_acc))

Epoch: 0; Loss: 0.690184593201; Training Acc: accuracy0.655502392344
Teting Acc: accuracy0.594594594595
Epoch: 100; Loss: 0.563165009022; Training Acc: accuracy0.655502392344
Teting Acc: accuracy0.594594594595
Epoch: 200; Loss: 0.393676787615; Training Acc: accuracy0.655502392344
Teting Acc: accuracy0.594594594595
Epoch: 300; Loss: 0.193265274167; Training Acc: accuracy0.655502392344
Teting Acc: accuracy0.594594594595
Epoch: 400; Loss: 0.110384240746; Training Acc: accuracy0.655502392344
Teting Acc: accuracy0.594594594595
Epoch: 500; Loss: 0.108408384025; Training Acc: accuracy0.655502392344
Teting Acc: accuracy0.594594594595
Epoch: 600; Loss: 0.084448993206; Training Acc: accuracy0.655502392344
Teting Acc: accuracy0.594594594595
Epoch: 700; Loss: 0.0514304675162; Training Acc: accuracy0.655502392344
Teting Acc: accuracy0.594594594595
Epoch: 800; Loss: 0.345853567123; Training Acc: accuracy0.655502392344
Teting Acc: accuracy0.594594594595
Epoch: 900; Loss: 0.0508930832148; Training Acc

In [21]:
acc.get()

AttributeError: 'float' object has no attribute 'get'

In [ ]:
acc = mx.metric.Accuracy()
for data, label in test_data:
    #acc.reset()
    data = data.as_in_context(mx.cpu())
    label = label.as_in_context(mx.cpu())
    output = net(data)
    acc.update(preds = output, labels = label)
    


In [ ]:
acc.get()